# 0. Применение метода KNN для решения задачи классификации

Набор данных получен в результате переписи населения 1994 года и содержит информацию о некотором количестве людей, проживающих в США. Задача состоит в том, чтобы предсказать, зарабатывает человек более $50к в год или нет. Список признаков:

*   <b>age</b>: возраст человека.
*   <b>workclass</b>: статус занятости.
*   <b>fnlwgt</b>: количество людей, которое, по мнению переписи, представляет запись.
*   <b>education</b>: высший уровень образования, достигнутый человеком.
*   <b>education-num</b>: высший уровень образования, достигнутый человеком в числовой форме.
*   <b>marital-status</b>: семейное положение человека.
*   <b>occupation</b>: общий род занятий человека.
*   <b>relationship</b>: представляет то, чем этот человек является по отношению к другим (перекликается с признаком <b>marital-status</b>).
*   <b>race</b>: раса.
*   <b>sex</b>: пол.
*   <b>capital-gain</b>: прирост капитала.
*   <b>capital-loss</b>: убыток капитала.
*   <b>hours-per-week</b>: число рабочих часов в неделю.
*   <b>native-country</b>: страна происхождения.
*   <b>the label</b>: отклик -- зарабатывает больше $50к или меньше.



## Немного про метрики качества модели

Для начала рассмотрим так называемую матрицу ошибок (confusion matrix)
-- способ разделить объекты на $4$ группы в зависимости от комбинации
истинного класса и ответа классификатора:

-   TP (True Positives) -- верно классифицированные объекты, исходно     относящиеся к классу "$+1$";

-   TN (True Negatives) -- верно классифицированные объекты, исходно     относящиеся к классу "$-1$";

-   FN (False Negatives) -- неверно классифицированные объекты, исходно     относящиеся к классу "$+1$" (ошибка I рода);

-   FP (False Positives) -- неверно классифицированные объекты, исходно     относящиеся к классу "$-1$" (ошибка II рода).

Обычно, конечно, оперируют не абсолютными показателями, а относительными
-- долями (rates), находящимися в диапазоне от $0$ до $1$:

-   доля правильных ответов классификатора (иногда -- точность): $$\mathsf{Accuracy} = \frac{TP + TN}{TP + FP + FN + TN}.$$ Эта величина показывает отношение количества верно классифицированных объектов к общему количеству классифицируемых объектов и, грубо говоря, оценивает вероятность случайному объекту быть правильно классифицированным.

-   доля истинно положительных примеров -- True Positives Rate (TPR) или     Sensitivity (чувствительность) или Recall: $$\mathsf{T P R}=\frac{T P}{T P+F N}.$$ Эта величина показывает отношение количества верно классифицированных объектов, относящихся к классу "$+1$", к общему количеству объектов класса "$+1$". Иными словами -- это оценка вероятности, что объект, относящийся к классу "$+1$" будет классифицирован корректно.

-   доля ложно положительных примеров обозначается как -- False Positives Rate (FPR): $$\mathsf{F P R}=\frac{F P}{FP + TN}.$$ Величина показывает отношение количества неверно классифицированных объектов, относящихся к классу "$-1$", к общему количеству объектов класса "$-1$", или оценивает вероятность, что объект, относящийся к классу "$-1$", будет классифицирован неверно.

-   Специфичность (Specificity) или True Negatives Rate (TNR): $$\mathsf{TNR} = 1 - \mathsf{F P R} =\frac{T N}{T N+F P}.$$ Величина показывает отношение количества верно классифицированных объектов, относящихся к классу "$-1$", к общему количеству объектов класса "$-1$", или оценивает вероятность, что объект, относящийся к классу "$-1$", будет классифицирован верно.

-   Precision (точность): $$\mathsf{Precision} =\frac{TP}{TP + FP}.$$ Величина показывает, какая доля объектов, отнесенных классификатором к классу "$+1$", действительно относится к этому классу.

Естественно возникает вопрос, нет ли какого-то обобщающего критерия,
который может характеризовать качество построенной модели. Один из них --
так называемая $F$-мера ($F_1$-мера, $F$ score, $F_1$ score)
определяется следующим соотношением:
$$F = F_1 = 2 \cdot \frac{\mathsf{Precision} \cdot \mathsf{Recall}}{\mathsf{Precision} + \mathsf{Recall}}.$$

**Замечание**. *$F$-мера является средним гармоническим величин
$\mathsf{Precision}$ и $\mathsf{Recall}$ и заключена в диапазоне
$[0, 1]$. Среднее гармоническое обладает важным свойством: оно близко к
нулю, если хотя бы один из аргументов близок к нулю. Поэтому оно является
куда более предпочтительным, чем, скажем, среднее арифметическое: если
алгоритм относит все объекты к положительному классу, то
$\mathsf{Recall}= 1$, а $\mathsf{Precision}$, скорее всего, будет
небольшим. Но тогда среднее арифметическое будет больше, чем $0.5$, что,
конечно, никуда не годится.*

## Импорт библиотек и чтение набора данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
import seaborn as sns
from matplotlib import pyplot as plt

Считайте набор данных в датафрейм. Исходя из описания признаков можно сразу избавиться от признаков <code>education</code> и <code>marital-status</code>. Удалите соответствующие колонки из набора данных.

In [ ]:
df = pd.read_csv("adult_data_train.csv")

In [ ]:
df = df.drop(columns = ['education', 'marital-status'])
df.head()

,age,workclass,fnlwgt,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,32,Private,37210,13,Exec-managerial,Husband,White,Male,0,0,45,United-States,1
1,43,Private,101950,14,Exec-managerial,Not-in-family,White,Female,0,0,45,United-States,0
2,20,?,122244,9,?,Not-in-family,White,Female,0,0,28,United-States,0
3,40,Local-gov,24763,10,Transport-moving,Unmarried,White,Male,6849,0,40,United-States,0
4,24,Private,113936,13,Prof-specialty,Own-child,White,Male,0,0,40,United-States,0


Определите количество числовых и нечисловых признаков.

In [ ]:
numerical_features = df.select_dtypes(include=[int, float])
categorical_features = df.select_dtypes(include=[object])

# 1. Построение базовой модели

В качестве первого приближения имеет смысл построить модель классификации, опираясь исключительно на числовые признаки.

Отберите из набора данных только числовые признаки. При помощи <code>train_test_split()</code> разбейте набор данных на обучающую и тестовую выборки <b>с параметрами, указанными в вашем задании</b>. Используйте стратификацию по колонке <code>label</code>.

Вычислите выборочное среднее колонки <code>fnlwgt</code> тренировочного набора данных.

Обучите модель <code>KNeighborsClassifier()</code> с параметрами по умолчанию на тренировочных данных.

Как видно из предыдущего пункта, в наборе данных наблюдается явный дисбаланс представителей классов. Это следует учесть при оценке модели. Вычислите <code>f1_score</code> модели на тестовых данных (рекомендуем использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html">соответствующую функцию</a> с параметрами по умолчанию.

В качестве альтернативы можно использовать так называемый <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html"><code>classification_report()</code></a>, где приведены сразу несколько метрик (не стоит забывать про параметр <code>digits</code>).

In [ ]:
X = numerical_features.drop(columns=['label'])
y = df['label']  # Целевая переменная
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41, stratify=y)

In [ ]:
mean_fnlwgt = X_train['fnlwgt'].mean()
print(f"Выборочное среднее fnlwgt: {mean_fnlwgt:.3f}")

Выборочное среднее fnlwgt: 190199.121


In [ ]:
# Обучение модели KNeighborsClassifier с параметрами по умолчанию
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred = knn_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"f1_score модели на тестовых данных: {f1:.3f}")

f1_score модели на тестовых данных: 0.379


Стоит помнить, что KNN является метрическим классификатором, поэтому значения признаков перед обучением модели следует нормировать.

Обучите преобразование <code>MinMaxScaler()</code> на тренировочном наборе данных и примените его для тренировочных и тестовых данных.

Вычислите выборочное среднее колонки <code>fnlwgt</code> тренировочного набора данных после нормировки.

Заново обучите и оцените модель на преобразованных данных. Вычислите <code>f1_score()</code> модели.

In [ ]:
# Разделение данных на обучающий и тестовый наборы с учетом стратификации
X = numerical_features.drop(columns=['label'])  # Все числовые признаки, кроме 'label'
y = df['label']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41, stratify=y)

# Создание и обучение MinMaxScaler на тренировочных данных
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Вычисление выборочного среднего fnlwgt после нормировки
mean_fnlwgt_normalized = X_train_normalized[:, X.columns.get_loc('fnlwgt')].mean()
print(f"Выборочное среднее fnlwgt после нормировки: {mean_fnlwgt_normalized:.3f}")

# Обучение модели KNeighborsClassifier с параметрами по умолчанию на нормированных данных
knn_model_normalized = KNeighborsClassifier()
knn_model_normalized.fit(X_train_normalized, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred_normalized = knn_model_normalized.predict(X_test_normalized)
f1_normalized = f1_score(y_test, y_pred_normalized)
print(f"f1_score модели на нормированных тестовых данных: {f1_normalized:.3f}")

Выборочное среднее fnlwgt после нормировки: 0.121
f1_score модели на нормированных тестовых данных: 0.505


Видно, что после линейной нормировки качество выросло.

<b>Важно: </b>На дальнейших этапах подразумевается использование линейной нормировки непосредственно перед обучением без дополнительных напоминаний.

# 2. Работа с нечисловыми признаками

## Визуализация

Для дальнейшего улучшения качества модели имеет смысл задействовать нечисловые признаки исходного датасета (без колонок <code>education</code> и <code>marital-status</code>).

Постройте гистограммы, иллюстрирующие частоту того или иного значения по каждому нечисловому признаку, например, при помощи <code>sns.barplot()</code>.

## Удаление пропущенных значений

Определите число строк исходного набора данных (без колонок <code>education</code> и <code>marital-status</code>), в которых присутствует хотя бы одно пропущенное значение.

In [ ]:
df = pd.read_csv("adult_data_train.csv")
df = df.drop(columns = ['education', 'marital-status'])
print(df.shape)

(26048, 13)


In [ ]:
df.replace("?", None, inplace=True)
df = df.dropna()
print(df.shape)

(24134, 13)


Видно, что в датасете содержится менее 10% строк, содержащих пропуски. Выкидывать такое количество строк — не очень хорошее дело, но почему бы не попробовать обойтись без них.

Удалите строки, содеражащие пропуски. Произведите <code>one-hot</code> кодировние нечисловых признаков, например, с помощью <code>pd.get_dummies(drop_first=True)</code>.

Введите число полученных признаков.

In [ ]:
categorical_features = df.select_dtypes(include=['object'])
print(categorical_features.shape)

(24134, 6)


In [ ]:
categorical_encoded = pd.get_dummies(categorical_features, drop_first=True)
df = df.drop(categorical_features.columns, axis=1)
df = pd.concat([df, categorical_encoded], axis=1)
df.shape

(24134, 76)

Обучите модель классификации аналогично тому, как это было проделано для базовой модели. Вычислите <code>f1_score()</code> модели.



In [ ]:
# Разделение данных на обучающий и тестовый наборы с учетом стратификации
X = df.drop(columns=['label'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41, stratify=y)

# Создание и обучение MinMaxScaler на тренировочных данных
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Обучение модели KNeighborsClassifier с параметрами по умолчанию на нормированных данных
knn_model_normalized = KNeighborsClassifier()
knn_model_normalized.fit(X_train_normalized, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred_normalized = knn_model_normalized.predict(X_test_normalized)
f1_normalized = f1_score(y_test, y_pred_normalized)
print(f"f1_score модели на нормированных данных после one-hot кодирования: {f1_normalized:.3f}")

f1_score модели на нормированных данных после one-hot кодирования: 0.600


Точность модели увеличилась по сравнению с моделью, которая использовала только числовые признаки.

## Заполнение пропущенных значений

Возможно точность еще повысится, если задействовать строки с пропущенными значениями. Используя исходный датасет (без колонок <code>education</code> и <code>marital-status</code>), заполните пропуски самым часто встречающимся значением в рамках столбца.

In [ ]:
df = pd.read_csv("adult_data_train.csv")
df = df.drop(columns = ['education', 'marital-status'])

In [ ]:
df.replace("?", None, inplace=True)
df = df.fillna(df.mode().iloc[0])

In [ ]:
df = df.fillna(df.mode().iloc[0])

Далее по уже знакомому сценарию: <code>one-hot</code>, <code>split</code>, <code>scaling</code>, обучение и оценка.

Вычислите <code>f1_score()</code> модели.

In [ ]:
categorical_features = df.select_dtypes(include=['object'])
categorical_encoded = pd.get_dummies(categorical_features, drop_first=True)
# Удалите исходные нечисловые признаки
df = df.drop(categorical_features.columns, axis=1)
# Объедините числовые признаки и закодированные нечисловые признаки
df = pd.concat([df, categorical_encoded], axis=1)

In [ ]:
df.shape

(26048, 77)

In [ ]:
# Разделение данных на обучающий и тестовый наборы с учетом стратификации
X = df.drop(columns=['label'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41, stratify=y)

# Создание и обучение MinMaxScaler на тренировочных данных
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Обучение модели KNeighborsClassifier с параметрами по умолчанию на нормированных данных
knn_model_normalized = KNeighborsClassifier()
knn_model_normalized.fit(X_train_normalized, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred_normalized = knn_model_normalized.predict(X_test_normalized)
f1_normalized = f1_score(y_test, y_pred_normalized)
print(f"f1_score модели на нормированных данных после one-hot кодирования: {f1_normalized:.3f}")

f1_score модели на нормированных данных после one-hot кодирования: 0.610


## Проклятие размерности

В последнем пункте был получен набор данных, содержащий 76 признаков (кстати, попробуйте объяснить, почему в случае удаления строк число признаков в итоге оказалось равным 75), что является достаточным для того, чтобы столкнуться с так называемым проклятием размерности.

Для того, чтобы классификатор давал более качественные результаты, имеет смысл более внимательно и вдумчиво поработать с признаками с учетом проклятия размерности. Например, вернуть в рассмотрение признаки <code>education-num</code> и <code>marital-status</code>. А также более глубоко вникнуть в саму природу признаков.

---

# 1

In [ ]:
df_train = pd.read_csv("adult_data_train.csv")
df_train = df_train.drop(columns = ['education', 'marital-status'])


df_train.replace("?", None, inplace=True)
df_train = df_train.fillna(df_train.mode().iloc[0])

categorical_features = df_train.select_dtypes(include=['object'])
categorical_encoded = pd.get_dummies(categorical_features, drop_first=True)

df_train = df_train.drop(categorical_features.columns, axis=1)
df_train = pd.concat([df_train, categorical_encoded], axis=1)
# df_train = df_train.drop(columns = ['native-country_Holand-Netherlands'])
df_train.shape

(26048, 76)

In [ ]:
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']

In [ ]:
X_test = pd.read_csv("adult_data_reserved.csv")
X_test = X_test.drop(columns = ['education', 'marital-status'])


X_test.replace("?", None, inplace=True)
X_test = X_test.fillna(X_test.mode().iloc[0])

categorical_features = X_test.select_dtypes(include=['object'])
categorical_encoded = pd.get_dummies(categorical_features, drop_first=True)

X_test = X_test.drop(categorical_features.columns, axis=1)
X_test = pd.concat([X_test, categorical_encoded], axis=1)
X_test.shape

(6513, 75)

In [ ]:
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Обучение модели KNeighborsClassifier с параметрами по умолчанию на нормированных данных
knn_model_normalized = KNeighborsClassifier()
knn_model_normalized.fit(X_train_normalized, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred_normalized = knn_model_normalized.predict(X_test_normalized)
predictions_list = y_pred_normalized.tolist()
print(predictions_list)

[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

0.617697300991

# 2

In [ ]:
df_train = pd.read_csv("adult_data_train.csv")
df_train = df_train.drop(columns=['education', 'marital-status'])
df_train.replace("?", None, inplace=True)
df_train = df_train.fillna(df_train.mode().iloc[0])
df_train.shape

(26048, 13)

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_features = df_train.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])
df_train.shape

(26048, 13)

In [ ]:
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']

X_test = pd.read_csv("adult_data_reserved.csv")
X_test = X_test.drop(columns = ['education', 'marital-status'])


X_test.replace("?", None, inplace=True)
X_test = X_test.fillna(X_test.mode().iloc[0])
X_test.shape

(6513, 12)

In [ ]:
categorical_features = X_test.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    X_test[column] = label_encoder.fit_transform(X_test[column])
X_test.shape

(6513, 12)

In [ ]:
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Обучение модели KNeighborsClassifier с параметрами по умолчанию на нормированных данных
knn_model_normalized = KNeighborsClassifier()
knn_model_normalized.fit(X_train_normalized, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred_normalized = knn_model_normalized.predict(X_test_normalized)
predictions_list = y_pred_normalized.tolist()
print(predictions_list)

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

Ваш результат: 0.620995228357

# 3

В 3 оставлял ['education', 'marital-status']). Стало хуже

Ваш результат: 0.618870476835

# 4

Hyperparameters tuning

In [ ]:
df = pd.read_csv("adult_data_train.csv")
df = df.drop(columns = ['education', 'marital-status'])
df.replace("?", None, inplace=True)
df = df.fillna(df.mode().iloc[0])
categorical_features = df.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    df[column] = label_encoder.fit_transform(df[column])
df.shape

(26048, 13)

In [ ]:
X = df.drop(columns=['label'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41, stratify=y)

# Создание и обучение MinMaxScaler на тренировочных данных
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'p': [1, 2],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40]
}



grid_search = GridSearchCV(
    KNeighborsClassifier(),
    param_grid,
    cv=5,
    scoring='f1_macro'
)

# Perform the grid search on the normalized training data
grid_search.fit(X_train_normalized, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_knn_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", best_params)

# Train the best model on the entire training dataset
best_knn_model.fit(X_train_normalized, y_train)

# Predict on the test data using the best model
y_pred_best = best_knn_model.predict(X_test_normalized)

Best Hyperparameters: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}


ValueError: ignored

In [ ]:
f1_normalized = f1_score(y_test, y_pred_best)
print(f"f1_score модели на нормированных данных после one-hot кодирования: {f1_normalized:.3f}")

f1_score модели на нормированных данных после one-hot кодирования: 0.612


# 4 try

In [ ]:
df_train = pd.read_csv("adult_data_train.csv")
df_train = df_train.drop(columns=['education', 'marital-status'])
df_train.replace("?", None, inplace=True)
df_train = df_train.fillna(df_train.mode().iloc[0])
df_train.shape

(26048, 13)

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_features = df_train.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])
df_train.shape

(26048, 13)

In [ ]:
X_train = df_train.drop(columns=['label'])
y_train = df_train['label']

X_test = pd.read_csv("adult_data_reserved.csv")
X_test = X_test.drop(columns = ['education', 'marital-status'])


X_test.replace("?", None, inplace=True)
X_test = X_test.fillna(X_test.mode().iloc[0])
X_test.shape

(6513, 12)

In [ ]:
categorical_features = X_test.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    X_test[column] = label_encoder.fit_transform(X_test[column])
X_test.shape

(6513, 12)

In [ ]:
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Обучение модели KNeighborsClassifier с параметрами по умолчанию на нормированных данных
knn_model_normalized = KNeighborsClassifier(algorithm = 'auto', leaf_size = 10, n_neighbors = 19, p = 1, weights='uniform', metric='manhattan')
knn_model_normalized.fit(X_train_normalized, y_train)

# Предсказание на тестовых данных и вычисление f1_score
y_pred_normalized = knn_model_normalized.predict(X_test_normalized)
predictions_list = y_pred_normalized.tolist()
print(predictions_list)

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

Ваш результат: 0.625220770046